BY: **RIYA JOSHI**

EMAIL: riya.joshi@somaiya.edu



---


### **Basic idea behind SVM(Support Vector Machine) algorithm**:

*   SVM (Support Vector Machine) is a supervised machine learning algorithm that can be used for both classification and regression tasks.
*   The algorithm works by finding a hyperplane (or decision boundary) which have the following properties:
  - It creates separation between data points of two classes with a maximum margin
  - Its equation (w.x + b = 0) yields a value ≥ 1 for examples from +ve class and ≤-1 for examples from -ve class
  
* It finds this hyperplane by finding the optimal values w (weights/normal) and b (intercept) which define this hyperplane. The optimal values are found by minimizing a cost function. 

* The data points above or on the hyperplane are classified as class +1 and the points be low are classified as class -1. In other words, the equation (w.x+b=0) yields a value ≥ 1 for +ve class and ≤ -1 for -ve class.


### **Applications of SVM**:

* Face detection 
* Text and hypertext categorization 
* Classification of images 
* Bioinformatics
* Protein fold and remote homology detection 
* Handwriting recognition 
* Generalized predictive control(GPC) 





---




In [53]:
# Importing required libraries
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [52]:
df = pd.read_csv('players_20.csv') #importing dataset
df.head() # displaying first five records

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,player_traits,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,24-06-1987,170,72,Argentina,FC Barcelona,94,94,95500000,565000,"RW, CF, ST",Left,5,4,4,Medium/Low,Messi,Yes,195800000.0,"#Dribbler, #Distance Shooter, #Crosser, #FK Sp...",RW,10.0,NaN,01-07-2004,2021.0,NaN,NaN,87.0,92.0,92.0,96.0,39.0,66.0,"Beat Offside Trap, Argues with Officials, Earl...",88,95,70,92,88,97,93,94,92,96,91,84,93,95,95,86,68,75,68,94,48,40,94,94,75,96,33,37,26,6,11,15,14,8
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,05-02-1985,187,83,Portugal,Juventus,93,93,58500000,405000,"ST, LW",Right,5,4,5,High/Low,C. Ronaldo,Yes,96500000.0,"#Speedster, #Dribbler, #Distance Shooter, #Acr...",LW,7.0,NaN,10-07-2018,2022.0,LS,7.0,90.0,93.0,82.0,89.0,35.0,78.0,"Long Throw-in, Selfish, Argues with Officials,...",84,94,89,83,87,89,81,76,77,92,89,91,87,96,71,95,95,85,78,93,63,29,95,82,85,95,28,32,24,7,11,15,14,11
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,27,05-02-1992,175,68,Brazil,Paris Saint-Germain,92,92,105500000,290000,"LW, CAM",Right,5,5,5,High/Medium,Neymar,Yes,195200000.0,"#Speedster, #Dribbler, #Playmaker , #Crosser,...",CAM,10.0,NaN,03-08-2017,2022.0,LW,10.0,91.0,85.0,87.0,95.0,32.0,58.0,"Power Free-Kick, Injury Free, Selfish, Early C...",87,87,62,87,87,96,88,87,81,95,94,89,96,92,84,80,61,81,49,84,51,36,87,90,90,94,27,26,29,9,9,15,15,11
3,200389,https://sofifa.com/player/200389/jan-oblak/20/...,J. Oblak,Jan Oblak,26,07-01-1993,188,87,Slovenia,Atlético Madrid,91,93,77500000,125000,GK,Right,3,3,1,Medium/Medium,Normal,Yes,164700000.0,NaN,GK,13.0,NaN,16-07-2014,2023.0,GK,1.0,NaN,NaN,NaN,NaN,NaN,NaN,"Flair, Acrobatic Clearance",13,11,15,43,13,12,13,14,40,30,43,60,67,88,49,59,78,41,78,12,34,19,11,65,11,68,27,12,18,87,92,78,90,89
4,183277,https://sofifa.com/player/183277/eden-hazard/2...,E. Hazard,Eden Hazard,28,07-01-1991,175,74,Belgium,Real Madrid,91,91,90000000,470000,"LW, CF",Right,4,4,4,High/Medium,Normal,Yes,184500000.0,"#Speedster, #Dribbler, #Acrobat",LW,7.0,NaN,01-07-2019,2024.0,LF,10.0,91.0,83.0,86.0,94.0,35.0,66.0,"Beat Offside Trap, Selfish, Finesse Shot, Spee...",81,84,61,89,83,95,83,79,83,94,94,88,95,90,94,82,56,84,63,80,54,41,87,89,88,91,34,27,22,11,12,6,8,8


In [54]:
# replacing null values with 0
df = df.fillna(value= 0)

In [55]:
# Shuffling dataset ( to remove any biases in the dataset if any )
df = df.sample(frac=1)

In [56]:
new_df=df[['overall', 'potential', 'shooting', 'pace', 'passing', 'skill_ball_control', 'physic', 'preferred_foot']]
new_df.head()

,overall,potential,shooting,pace,passing,skill_ball_control,physic,preferred_foot
16918,56,66,36.0,62.0,49.0,48,57.0,Right
3835,72,72,47.0,67.0,67.0,70,73.0,Right
7194,68,69,62.0,69.0,65.0,68,64.0,Right
9601,66,66,0.0,0.0,0.0,13,0.0,Right
4221,71,75,68.0,79.0,68.0,70,55.0,Right


In [57]:
# converting type from object to string to be able to apply Label Encoder
new_df['preferred_foot']=new_df['preferred_foot'].astype(dtype='string',copy=True)

In [58]:
# converting catagorical column 'preferred_foot' to numerical column by LabelEncoding
new_df['preferred_foot'] = LabelEncoder().fit_transform(new_df['preferred_foot'])

In [59]:
# changing data type of whole dataset to int
new_df = new_df.astype(int)

In [60]:
# splitting dataset into 70:30 ratio

# Defining train size
train_size = int(0.7 * len(new_df))

# Splitting dataset
train_set = new_df[:train_size]
test_set = new_df[train_size:]

In [61]:
# separating train_set into X and Y
X_train=train_set.drop('preferred_foot', axis=1)
y_train=train_set['preferred_foot']

# separating test_set into X and Y
X_test=test_set.drop('preferred_foot', axis=1)
y_test=test_set['preferred_foot']

In [63]:
class SVM:

	def __init__(self, kernel = "linear", learning_rate = 1e-4, regularization_strength = 1.0, max_iter = 2000):
   
    """ There are many different types of Kernels , for ex: linear, polynomial, Sigmoid, and Radial Basis Function (RBF). 
        Normally, the kernel is linear, and we get a linear SVM classifier. """
 
		self.num_feats = int
		self.train_size = int
		self.weights = np.array 
		self.y_train = np.array 
		self.input_matrix = np.array

		self.kernel = kernel
		self.kernel_matrix = np.array
		self.support_vectors = np.array
		self.learning_rate = learning_rate 	# Learning rate for gradient descent
		self.regularization_strength = regularization_strength 	# Regularization parameter, to control bias-variance tradeoff
		self.max_iter = max_iter	# Maximum Number of iterations to run gradient descent
		self.cost_threshold = 0.1 * learning_rate  # stopping criterion for gradient descent

	def fit(self, X, y):

		""" Adjust weights to training data """

		self.train_size = X.shape[0]
		self.num_feats = X.shape[1]
		self.input_matrix = np.append(X, np.ones(self.train_size).reshape(-1, 1), axis = 1)   #Add Column with Ones for intercept term 
		self.y_train = np.where(y == 0, -1, 1)
		self.weights = np.zeros(self.num_feats + 1) #Extra +1 for the intercept

		# optimize weights
		prev_cost = float("inf")
		for i in range(self.max_iter):
			cost = self._update_weights()
			
			if i%100 ==0 or i == self.max_iter:
				print("Cost after {} iterations is: {}".format(i, cost))
			if abs(prev_cost -cost) < self.cost_threshold*prev_cost:
				print("Cost after {} iterations is: {}".format(i, cost))
				break
			prev_cost = cost

	def _update_weights(self):

		"""
			Cost Function:
				l(w) = sum(max(0, 1 - y(wX + b))) + (lambda/2)(||w||)^2
				First Term is Hinge Loss, Second is Regularization term
			Gradient:
			    delta_w = dl/dw = (1/n) * ( if y(wX+b) < 1: -yX + lambda*w else: lambda*w )
			Gradient Descent
				w = w - (learning_rate * delta_w)
		"""
		y_pred = (self.weights * self.input_matrix).sum(axis = 1) # y_pred = wX+b


		dist = (1 - (self.y_train * y_pred))
		dist[dist < 0] = 0
		hinge_loss = np.sum(dist)/self.train_size
		regularization_term = (1/2) * self.regularization_strength * (np.dot(self.weights, self.weights))
		cost = hinge_loss  + regularization_term

		delta_w =  self.regularization_strength * (self.weights)

		for y, yhat, X in zip(self.y_train, y_pred, self.input_matrix):

			if y*yhat < 1:
				delta_w -= y*X

		delta_w /= self.train_size
		self.weights = self.weights - (self.learning_rate * delta_w)

		return cost

	def predict(self, X):

		""" Make predictions on given X using trained model """

		size = X.shape[0]
		X = np.append(X, np.ones(size).reshape(-1, 1), axis = 1)

		y_pred = np.sign((self.weights * X).sum(axis = 1))

		y_pred[np.where(y_pred == -1)] = 0.0

		return y_pred 

In [64]:
svm_clf = SVM(learning_rate = 1e-5, regularization_strength = 10.5)

svm_clf.fit(X_train, y_train)

Cost after 0 iterations is: 1.0
Cost after 100 iterations is: 0.5099058989896437
Cost after 200 iterations is: 0.5021267539047026
Cost after 300 iterations is: 0.5014371255356304
Cost after 400 iterations is: 0.5010649754271096
Cost after 500 iterations is: 0.5008182722197009
Cost after 600 iterations is: 0.5006418361333153
Cost after 700 iterations is: 0.500513554633431
Cost after 800 iterations is: 0.5004182718237089
Cost after 900 iterations is: 0.5003496743480967
Cost after 944 iterations is: 0.5003264946774291


In [65]:
print("Coefficients: {}".format(svm_clf.weights[:-1]))

Coefficients: [ 0.00675676  0.00949542  0.00049114  0.00103493 -0.00418084 -0.00086468
  0.00151727]


In [66]:
print("Intercept: {}".format(svm_clf.weights[-1]))

Intercept: 0.0003360167426367691


In [67]:
def accuracy_score(y_true, y_pred):

	"""	score = (y_true - y_pred) / len(y_true) """

	return round(float(sum(y_pred == y_true))/float(len(y_true)) * 100 ,2)

In [68]:
print("Train Accuracy: {}".format(accuracy_score(y_train, svm_clf.predict(X_train))))
print("Test Accuracy: {}".format(accuracy_score(y_test, svm_clf.predict(X_test))))

Train Accuracy: 76.08
Test Accuracy: 77.06
